In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import re
import requests 
import pandas as pd
import pickle

In [3]:
driver = webdriver.Chrome('Users/wongh/Downloads/chromedriver_win32/chromedriver.exe')

C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\3074833328.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('Users/wongh/Downloads/chromedriver_win32/chromedriver.exe')


In [4]:
def get_html_table_as_df(table_xpath,df_to_save):
    '''This saves a html table as a df. 
    Rmb to create an empty df to save the data to before executing this function. 
    If the df to save the data to is not empty, this function appends cases'''
      
    # Use find_element_by_xpath method to locate an element
    table_html = driver.find_element(by=By.XPATH, value=table_xpath).get_attribute('outerHTML')
    
    data_table = pd.read_html(table_html)
    df = data_table[0]
    if df_to_save.empty:
        df_to_save = data_table[0]
    else: df_to_save = df_to_save.append(data_table[0])
    return df_to_save

In [5]:
def data_gov_table_scrapper(url, table_xpath, number_of_pages_to_scrap, next_button_xpath, df_to_save):
    '''This  scraps a table from data.gov.sg and returns a df. 
    Rmb to create an empty df to save the data to before executing this function'''
    driver.get(url)
    time.sleep(2) ###to ensure entire page has been loaded 
    for page in range(number_of_pages_to_scrap): 
        if page == 0:
            driver.switch_to.frame(0) 
            ###the table of interest is in an iframe
            ###there is only 1 frame on the page
        df_to_save = get_html_table_as_df(table_xpath,df_to_save)
        if page == number_of_pages_to_scrap-1: 
            print('Complete!')
            break 
        else:
            next_page = driver.find_element(by=By.XPATH, value=next_button_xpath)
            next_page.send_keys(Keys.RETURN)
    return df_to_save

In [6]:
table_xpath = "//*[@id='resource_table']"
next_page_xpath = "//*[@id='resource_table_next']/a"

dengue_cases_url = 'https://data.gov.sg/dataset/weekly-number-of-dengue-and-dengue-haemorrhagic-fever-cases'

df_dengue = pd.DataFrame() 
df_dengue = data_gov_table_scrapper(dengue_cases_url,table_xpath,54,next_page_xpath,df_dengue)

C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.ap

Complete!


C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])


In [7]:
df_dengue

,Year,Eweek (Week),Type dengue,Number
0,2018,1,Dengue,83.0
1,2018,1,DHF,0.0
2,2018,2,Dengue,68.0
3,2018,2,DHF,0.0
4,2018,3,Dengue,54.0
...,...,...,...,...
10,2014,51,DHF,0.0
11,2014,52,Dengue,188.0
12,2014,52,DHF,0.0
13,2014,53,Dengue,158.0


In [8]:
# Creating dengue.csv
df_dengue.to_csv('dengue.csv')

In [9]:
mss_url = 'http://www.weather.gov.sg/climate-historical-daily/'
driver.get(mss_url)
time.sleep(2)

In [10]:
def button_dropdown_option_loop(option_xpath_id, number_of_options, display_button_xpath, 
                                table_xpath, df_to_save):
    '''This function selects the button and clicks on each option in order, starting from the top.
    Diplay button is then clicked to display the table below, and saves the table as a df'''
    for option in range(1,number_of_options+1):
        driver.find_element(by=By.XPATH, value=option_xpath_id).click()
        driver.find_element(by=By.XPATH, value=option_xpath_id+'/ul/li['+str(option)+']/a').click()
        driver.find_element(by=By.XPATH, value=display_button_xpath).click()
        time.sleep(2)
        df_to_save = get_html_table_as_df(table_xpath, df_to_save) 
        time.sleep(1)
    return df_to_save

In [11]:
def button_dropdown_option(option_xpath_id, option_index, display_button_xpath):
    '''This function selects the button and clicks on a specific option.
    Index[1] is the first option from the top. 
    Diplay button is then clicked to display the table below'''
    driver.find_element(by=By.XPATH, value=option_xpath_id).click()
    driver.find_element(by=By.XPATH, value=option_xpath_id+'/ul/li['+str(option_index)+']/a').click()
    driver.find_element(by=By.XPATH, value=display_button_xpath).click()
    time.sleep(1)

In [12]:
month_option_xpath_id = '//*[@id="monthDiv"]'
display_button_xpath = '//*[@id="display"]'
year_option_xpath_id = '//*[@id="yearDiv"]'

table_xpath = '//*[@id="temp"]/h4[2]/div/div/table'

### we are gonna do this one at a time because:
### a) this page loads very slowly
### b) we need to add a year column
### c) random errors seem to be happening (but seem to have disappeared when I increased the timer)
### d) I don't know how to create a loop that creates variabe names dynamically (e.)

In [13]:
##2018 
df_temp_rain2018 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,2,display_button_xpath)
df_temp_rain2018 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2018)
df_temp_rain2018['Year'] = 2018

C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.ap

In [14]:
##2017 
df_temp_rain2017 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,3,display_button_xpath)
df_temp_rain2017 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2017)
df_temp_rain2017['Year'] = 2017

C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.ap

In [15]:
##2016
df_temp_rain2016 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,4,display_button_xpath)
df_temp_rain2016 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2016)
df_temp_rain2016['Year'] = 2016

C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.ap

In [16]:
##2015
df_temp_rain2015 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,5,display_button_xpath)
df_temp_rain2015 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2015)
df_temp_rain2015['Year'] = 2015

C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.ap

In [17]:
##2014
df_temp_rain2014 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,6,display_button_xpath)
df_temp_rain2014 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2014)
df_temp_rain2014['Year'] = 2014

C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.ap

In [18]:
df_temp_rain = df_temp_rain2018
df_temp_rain = df_temp_rain.append([df_temp_rain2017,df_temp_rain2016,df_temp_rain2015,df_temp_rain2014])

C:\Users\wongh\AppData\Local\Temp\ipykernel_13392\54678392.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_temp_rain = df_temp_rain.append([df_temp_rain2017,df_temp_rain2016,df_temp_rain2015,df_temp_rain2014])


In [19]:
with open('df_temp_rain2018.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2018, to_write)
with open('df_temp_rain2017.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2017, to_write)
with open('df_temp_rain2016.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2016, to_write)
with open('df_temp_rain2015.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2015, to_write)
with open('df_temp_rain2014.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2014, to_write)
with open('df_temp_rain.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain, to_write)

In [20]:
df_temp_rain


,Date,Daily Rainfall Total (mm),Highest 30-min Rainfall (mm),Highest 60-min Rainfall (mm),Highest 120-min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Year
0,1 Dec,16.4,10.0,13.2,15.2,26.4,31.0,23.4,6.6,31.5,2018
1,2 Dec,7.2,5.8,6.2,6.8,26.5,30.2,24.0,6.9,29.6,2018
2,3 Dec,3.0,2.4,2.6,2.6,27.4,31.7,24.2,6.0,31.5,2018
3,4 Dec,15.2,9.2,9.2,9.2,27.1,30.3,24.5,5.2,31.5,2018
4,5 Dec,8.2,7.0,8.0,8.0,26.8,30.9,25.2,6.7,24.1,2018
...,...,...,...,...,...,...,...,...,...,...,...
26,27 Jan,6.6,4.6,6.0,6.6,27.1,32.0,25.0,7.9,28.8,2014
27,28 Jan,12.2,5.2,7.8,9.6,26.3,31.5,23.6,6.8,27.4,2014
28,29 Jan,0.0,0.0,0.0,0.0,27.7,33.2,24.9,8.3,31.0,2014
29,30 Jan,40.8,27.2,34.0,39.6,26.9,32.6,21.8,9.0,50.0,2014


In [21]:
# Creating temp_rain.csv
df_temp_rain.to_csv('temp_rain.csv')